# Genetic diseases alter liver metabolism (From Chapter 11)
Inborn errors in metabolic genes can cause metabolic conditions that affect the body's ability to properly process certain nutrients or substances. These conditions can have a wide range of metabolic consequences, including changes in energy metabolism, nutrient utilization, and hormone regulation. Since the activity of multiple genes that are commonly impacted by inborn errors of metabolism are included in CoreLiver, we can systematically assess the impact of such disorders on the liver metabolic network.

**Propionic acidemia** Propionic acidemia is caused by mutations in the genes that encode propionyl-CoA carboxylase (PPCOACm), which is a mitochondrial enzyme that converts Propinoyl-CoA (ppcoa) into (S)-Methyl\-malonyl-CoA (mmcoa\_\_S). Using the model we can ask how removing the function of this enzyme may change the ability of the liver to metabolize certain nutrients. This can simply be done by restricting all flux through PPCOACm and optimizing ATP production using each possible media metabolite as the sole nutrient source. This analysis shows that, without PPCOACm, the model cannot generate ATP from L-isoleucine (iso\_\_L), Propionic Acid (ppa), or L-methionine (met\_\_L). It is known that those with propionic acidemia cannot metabolize L-valine, L-isoleucine, L-threonine, L-methionine, and odd-chained fatty acids. CoreLiver modeling results accurately reflect these metabolic limitations in {numref}`propionic_acidemia`.

```{figure} ../images/pcca_disease.png
---
height: 500px
name: propionic_acidemia
---
ATP yield of common substrates with and without Propionyl-CoA carboxylase (PPCOACm)

```

```{note}
Use the launch buttons in the top right to regenerate the plot here (using Live Code) or open in a Binder
```

In [ ]:
import pandas as pd
import cobra
import seaborn as sns
from matplotlib import pyplot as plt
import math
from pathlib import Path

media = ['EX_lac__L_c', 'EX_pyr_c', 'EX_octa_c', 'EX_gln__L_c', 'EX_bhb_c',
             'EX_glu__L_c', 'EX_ser__L_c', 'EX_cys__L_c', 'EX_gly_c', 'EX_ala__L_c',
            'EX_etoh_c', 'EX_fru_c', 'EX_glc__D_c', 'EX_arg__L_c', 'EX_pro__L_c', 'EX_ile__L_c', 'EX_leu__L_c',
        'EX_asp__L_c', 'EX_asn__L_c', 'EX_ppa_c', 'EX_atp_c', 'EX_utp_c', 'EX_met__L_c',
         'EX_glyc_3octa_c', 'EX_glyc_c']

# ---------------------------------------
# Simulation: Run model for each media under both conditions
# ---------------------------------------
fluxes_with = {}
fluxes_without = {}
summary = []

model = cobra.io.load_json_model(list(Path('..').rglob('core_model.json'))[0])
model.reactions.EX_glc__D_c.knock_out()

for m in media:
    if m in model.reactions:
        # With PCCA:
        with model:
            model.reactions.get_by_id(m).lower_bound = -1
            sol = model.optimize()
            fluxes_with[m] = sol.fluxes.copy()
            summary.append(dict(media=m, atp=sol.objective_value, condition='with PCCA'))
        # Without PCCA:
        with model:
            model.reactions.get_by_id(m).lower_bound = -1
            model.reactions.PPCOACm.knock_out()
            sol = model.optimize()
            fluxes_without[m] = sol.fluxes.copy()
            summary.append(dict(media=m, atp=sol.objective_value, condition='without PCCA'))

# Create a DataFrame summary for the bar plot
fig, ax = plt.subplots(figsize=(10, 5))
ax = sns.barplot(data=pd.DataFrame(summary).sort_values('atp'), x='media', y='atp', hue='condition', ax=ax)
ax.tick_params(labelsize=15)
ax.tick_params(rotation=90, axis='x')
ax.set_ylabel(r'ATP synthesis ($\frac{mmol}{gDW \cdot hr}$)', size=20)
ax.set_xlabel('Nutrient', size=20)
ax.set_title('ATP synthesis with and without PCCA')
_ = ax.legend(title='Condition', title_fontsize=15, fontsize=15)

```{note}
Use the launch buttons in the top right to observe CoreLiver flux solutions for various nutrient sources with and without PCCA enzyme activity
```

In [1]:
from ipywidgets import Dropdown, RadioButtons, VBox, Output
from IPython.display import display
import pandas as pd
import cobra
from escher import Builder
import seaborn as sns
from matplotlib import pyplot as plt
import math
from pathlib import Path

# ---------------------------------------
# Escher Map: Create an Escher Builder for flux visualization
# ---------------------------------------
default_media = 'EX_glc__D_c'
map_json_loc = str(list(Path('..').rglob('core_map_v30.json'))[0])
builder = Builder(map_json=map_json_loc, 
                  reaction_data=fluxes_with[default_media])

builder.reaction_scale = [
    {'type': 'value', 'color': 'red', 'size': 20, 'value': -0.01},
    {'type': 'value', 'color': 'grey', 'size': 0, 'value': 0},
    {'type': 'value', 'color': 'red', 'size': 20, 'value': 0.01}
]

escher_out = Output()
with escher_out:
    display(builder)

# ---------------------------------------
# ipywidgets for interactive toggling
# ---------------------------------------
condition_widget = RadioButtons(options=['with PCCA', 'without PCCA'],
                                value='with PCCA', description='Condition:')
media_widget = Dropdown(options=sorted(list(fluxes_with.keys())),
                        value='EX_glc__D_c', description='Media:')

def update_outputs(change):
    # Get current selections
    cond = condition_widget.value
    med = media_widget.value

    # --- Update Escher map ---
    flux_data = fluxes_with[med] if cond == 'with PCCA' else fluxes_without[med]
    builder.reaction_data = flux_data


# Attach the update function to both widgets
condition_widget.observe(update_outputs, names='value')
media_widget.observe(update_outputs, names='value')

# ---------------------------------------
# Display all interactive elements in the notebook
# ---------------------------------------
ui = VBox([condition_widget, media_widget])
display(ui)
escher_out

AttributeError: module 'numpy' has no attribute 'float'.
`np.float` was a deprecated alias for the builtin `float`. To avoid this error in existing code, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations